# Quantitative Momentum Investing

Momentum investing is defined as investing in stocks that have increased in price the most. In this project, I will build a quantitative momentum investing strategy that selects the 50 stocks from S&P 500 with the highest price momentum and then calculate recommended trades for an equal-weight portfolio of the 50 selected stocks.

## Library Imports

Here, I import the open-source software libraries needed for the project.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
from statistics import mean
import xlsxwriter

## Import Stock List and API Token

In [2]:
stocks = pd.read_csv('sp500_constituents.csv')
stocks

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [3]:
stocks = stocks.replace('WLTW', 'WTW')
stocks = stocks.replace('VIAC', 'PARA')
from secrets import IEX_CLOUD_API_TOKEN

## First API Call

I will get the returns over the one year period for each stock available.

In [4]:
symbol = 'FB'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Meta Platforms Inc',
 'marketcap': 591560367563,
 'week52high': 395.54,
 'week52low': 207.88,
 'week52highSplitAdjustOnly': 395.81,
 'week52lowSplitAdjustOnly': 208.11,
 'week52change': -0.2216913600903573,
 'sharesOutstanding': 2359128546,
 'float': 0,
 'avg10Volume': 42940907,
 'avg30Volume': 53403289,
 'day200MovingAvg': 330.3,
 'day50MovingAvg': 290.38,
 'employees': 59527,
 'ttmEPS': 28.06,
 'ttmDividendRate': 0,
 'dividendYield': 0,
 'nextDividendDate': '',
 'exDividendDate': '',
 'nextEarningsDate': '2022-04-26',
 'peRatio': 7.494179597034607,
 'beta': 1.724925938194842,
 'maxChangePercent': 4.412802645662467,
 'year5ChangePercent': 0.549204698850382,
 'year2ChangePercent': -0.05517867037620861,
 'year1ChangePercent': -0.2353935918064346,
 'ytdChangePercent': -0.3959102516862533,
 'month6ChangePercent': -0.4228659688393842,
 'month3ChangePercent': -0.3979771204394814,
 'month1ChangePercent': -0.365228135274821,
 'day30ChangePercent': -0.3649673777338783,
 'day5C

## Parse API Call

In [5]:
data['year1ChangePercent']

-0.2353935918064346

In [6]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups = list(chunks(stocks['Symbol'], 100))

symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
for symbol_string in symbol_strings:
    print(symbol_string)
    
df_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,APA,AAPL,AMAT,APTV,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY,CERN
CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CTXS,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CPRT,GLW,CTVA,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISCA,DISCK,DISH,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DRE,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,LLY,EMR,ENPH,ETR,EOG,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FB,FAST,FRT,FDX,FIS,FITB,FRC,FE,FISV,FLT,FMC
F,FTNT,FTV,FBHS,FOXA,FOX,BEN,FCX,GPS,GRMN,IT,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HBI,HAS,HCA,PEAK,HSIC,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUM,HBAN,HII

## Add Data to DataFrame

In [7]:
df = pd.DataFrame(columns = df_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = df_columns
            ),
            ignore_index = True
        )

df
        

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,153.88,-0.155168,N/A
1,AOS,71.60,0.250230,N/A
2,ABT,120.06,-0.056055,N/A
3,ABBV,150.84,0.423289,N/A
4,ABMD,301.15,-0.073969,N/A
...,...,...,...,...
500,YUM,129.40,0.221092,N/A
501,ZBRA,430.88,-0.125772,N/A
502,ZBH,123.50,-0.229415,N/A
503,ZION,72.30,0.464281,N/A


## Remove Low-Momentum Stocks

1) Sort the stocks by One-Year Price Return

2) Drop all stocks outside the top 50

In [8]:
df.sort_values('One-Year Price Return', ascending = False, inplace = True)

df = df[:50]

df.reset_index(drop = True, inplace = True)

df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,56.34,1.807978,N/A
1,MRO,22.14,1.440742,N/A
2,NUE,127.48,1.222620,N/A
3,FANG,136.02,1.096826,N/A
4,EOG,114.57,0.954709,N/A
5,COP,91.58,0.953950,N/A
6,MCHP,73.25,0.903433,N/A
7,PXD,238.11,0.858716,N/A
8,FTNT,317.66,0.851037,N/A
9,CF,75.95,0.724898,N/A


## Calculate the Number of Shares to Buy

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Please enter the value of your portfolio: ')
    try:
        print(float(portfolio_size))
    except ValueError:
        print('Please enter a number.')
        portfolio_size = input('Please once again enter the value of your portfolio: ')
    
portfolio_input()

Please enter the value of your portfolio: 100000000
100000000.0


In [10]:
position_size = float(portfolio_size) / len(df.index)

for i in range(0, len(df)):
    df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / df.loc[i, 'Price'])

df

/Users/SONY/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,56.34,1.807978,35498
1,MRO,22.14,1.440742,90334
2,NUE,127.48,1.222620,15688
3,FANG,136.02,1.096826,14703
4,EOG,114.57,0.954709,17456
5,COP,91.58,0.953950,21838
6,MCHP,73.25,0.903433,27303
7,PXD,238.11,0.858716,8399
8,FTNT,317.66,0.851037,6296
9,CF,75.95,0.724898,26333


## Better Momentum Strategy

Better momentum strategies take into account **high-quality** and **low-quality** momentum stocks, where high-quality momentum stocks show "slow and steady" outperformance over a substantial period of time and low-quality momentum stocks surge upwards despite potentially not showing any momentum for a long time.

Low-quality stocks are less preferred because they are often caused by short-terms news that is unlikely to repeat in the future. In other words, upward momentum of low-quality stocks are often not replicable and sustainable.

High-quality stocks can be identified by there 1-month, 3-month, 6-month, and 1-year price returns.

Note: `hqm` stands for high-quality momentum.

In [11]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]

hqm_df = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'Number of Shares to Buy',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A'
                ],
                index = hqm_columns 
            ),
            ignore_index = True
        )

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,150.41,Number of Shares to Buy,-0.150011,N/A,-0.240833,N/A,-0.166719,N/A,-0.170700,N/A
1,AOS,74.80,Number of Shares to Buy,0.245187,N/A,0.021348,N/A,-0.137593,N/A,-0.115514,N/A
2,ABT,116.88,Number of Shares to Buy,-0.056563,N/A,-0.051565,N/A,-0.085003,N/A,-0.080572,N/A
3,ABBV,146.91,Number of Shares to Buy,0.429756,N/A,0.257338,N/A,0.248799,N/A,0.054573,N/A
4,ABMD,310.42,Number of Shares to Buy,-0.075394,N/A,-0.102234,N/A,-0.124404,N/A,0.021166,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,131.90,Number of Shares to Buy,0.216332,N/A,-0.052414,N/A,-0.000994,N/A,0.000032,N/A
501,ZBRA,422.95,Number of Shares to Buy,-0.123614,N/A,-0.249285,N/A,-0.321625,N/A,-0.190987,N/A
502,ZBH,123.64,Number of Shares to Buy,-0.235019,N/A,-0.163044,N/A,-0.062349,N/A,-0.044165,N/A
503,ZION,73.43,Number of Shares to Buy,0.450054,N/A,0.335546,N/A,0.076404,N/A,0.024764,N/A


## Calculate Momentum Percentiles

In [12]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_df.index:
    for time_period in time_periods:
        hqm_df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_df[f'{time_period} Price Return'], hqm_df.loc[row, f'{time_period} Price Return'])

        
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,150.41,Number of Shares to Buy,-0.150011,10.49505,-0.240833,6.930693,-0.166719,19.80198,-0.170700,3.366337
1,AOS,74.80,Number of Shares to Buy,0.245187,67.128713,0.021348,57.227723,-0.137593,27.326733,-0.115514,15.445545
2,ABT,116.88,Number of Shares to Buy,-0.056563,19.009901,-0.051565,38.019802,-0.085003,39.60396,-0.080572,27.722772
3,ABBV,146.91,Number of Shares to Buy,0.429756,89.50495,0.257338,90.891089,0.248799,98.019802,0.054573,89.90099
4,ABMD,310.42,Number of Shares to Buy,-0.075394,17.227723,-0.102234,25.742574,-0.124404,29.90099,0.021166,78.217822
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,131.90,Number of Shares to Buy,0.216332,63.366337,-0.052414,37.425743,-0.000994,58.613861,0.000032,70.29703
501,ZBRA,422.95,Number of Shares to Buy,-0.123614,13.267327,-0.249285,6.138614,-0.321625,3.564356,-0.190987,2.178218
502,ZBH,123.64,Number of Shares to Buy,-0.235019,5.148515,-0.163044,14.455446,-0.062349,45.346535,-0.044165,44.752475
503,ZION,73.43,Number of Shares to Buy,0.450054,90.49505,0.335546,94.059406,0.076404,76.633663,0.024764,80.19802


## Calculate HQM Score

To filter stocks.

The HQM score will be the arithmetic mean of the 4 momentum percentile scores.

In [13]:
for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,150.41,Number of Shares to Buy,-0.150011,10.49505,-0.240833,6.930693,-0.166719,19.80198,-0.170700,3.366337,10.148515
1,AOS,74.80,Number of Shares to Buy,0.245187,67.128713,0.021348,57.227723,-0.137593,27.326733,-0.115514,15.445545,41.782178
2,ABT,116.88,Number of Shares to Buy,-0.056563,19.009901,-0.051565,38.019802,-0.085003,39.60396,-0.080572,27.722772,31.089109
3,ABBV,146.91,Number of Shares to Buy,0.429756,89.50495,0.257338,90.891089,0.248799,98.019802,0.054573,89.90099,92.079208
4,ABMD,310.42,Number of Shares to Buy,-0.075394,17.227723,-0.102234,25.742574,-0.124404,29.90099,0.021166,78.217822,37.772277
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,131.90,Number of Shares to Buy,0.216332,63.366337,-0.052414,37.425743,-0.000994,58.613861,0.000032,70.29703,57.425743
501,ZBRA,422.95,Number of Shares to Buy,-0.123614,13.267327,-0.249285,6.138614,-0.321625,3.564356,-0.190987,2.178218,6.287129
502,ZBH,123.64,Number of Shares to Buy,-0.235019,5.148515,-0.163044,14.455446,-0.062349,45.346535,-0.044165,44.752475,27.425743
503,ZION,73.43,Number of Shares to Buy,0.450054,90.49505,0.335546,94.059406,0.076404,76.633663,0.024764,80.19802,85.346535


## Best 50 Momentum Stocks

Sort DataFrame according to HQM score.

In [14]:
hqm_df.sort_values('HQM Score', ascending = False, inplace = True)

hqm_df = hqm_df[:50]

hqm_df.reset_index(drop = True, inplace = True)

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRO,22.60,Number of Shares to Buy,1.491307,99.80198,1.003497,99.60396,0.367980,99.80198,0.129919,98.415842,99.405941
1,HAL,33.21,Number of Shares to Buy,0.667910,97.821782,0.740840,98.811881,0.423816,100.0,0.130643,98.613861,98.811881
2,DVN,55.65,Number of Shares to Buy,1.857853,100.0,1.199270,100.0,0.293517,98.811881,0.090376,95.247525,98.514851
3,PXD,244.53,Number of Shares to Buy,0.852477,98.613861,0.715441,98.415842,0.329110,99.60396,0.082871,94.455446,97.772277
4,OXY,39.68,Number of Shares to Buy,0.590712,95.841584,0.694649,98.217822,0.304150,99.207921,0.100752,97.029703,97.574257
5,EOG,114.96,Number of Shares to Buy,0.962486,99.207921,0.801202,99.009901,0.297833,99.009901,0.058383,90.891089,97.029703
6,SLB,41.02,Number of Shares to Buy,0.614278,96.831683,0.520705,97.623762,0.272865,98.613861,0.076854,93.465347,96.633663
7,FANG,132.20,Number of Shares to Buy,1.103048,99.405941,0.935268,99.207921,0.222290,96.237624,0.045857,88.712871,95.891089
8,XOM,77.50,Number of Shares to Buy,0.577627,95.445545,0.460206,96.633663,0.230275,97.227723,0.073245,93.069307,95.594059
9,MCK,276.38,Number of Shares to Buy,0.562599,94.851485,0.368841,95.049505,0.209333,95.841584,0.075955,93.267327,94.752475


## Number of Shares to Buy

In [15]:
portfolio_input()

Please enter the value of your portfolio: 100000000
100000000.0


In [16]:
position_size = float(portfolio_size) / len(hqm_df.index)

for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_df.loc[i, 'Price'])
    
hqm_df

/Users/SONY/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRO,22.60,88495,1.491307,99.80198,1.003497,99.60396,0.367980,99.80198,0.129919,98.415842,99.405941
1,HAL,33.21,60222,0.667910,97.821782,0.740840,98.811881,0.423816,100.0,0.130643,98.613861,98.811881
2,DVN,55.65,35938,1.857853,100.0,1.199270,100.0,0.293517,98.811881,0.090376,95.247525,98.514851
3,PXD,244.53,8178,0.852477,98.613861,0.715441,98.415842,0.329110,99.60396,0.082871,94.455446,97.772277
4,OXY,39.68,50403,0.590712,95.841584,0.694649,98.217822,0.304150,99.207921,0.100752,97.029703,97.574257
5,EOG,114.96,17397,0.962486,99.207921,0.801202,99.009901,0.297833,99.009901,0.058383,90.891089,97.029703
6,SLB,41.02,48756,0.614278,96.831683,0.520705,97.623762,0.272865,98.613861,0.076854,93.465347,96.633663
7,FANG,132.20,15128,1.103048,99.405941,0.935268,99.207921,0.222290,96.237624,0.045857,88.712871,95.891089
8,XOM,77.50,25806,0.577627,95.445545,0.460206,96.633663,0.230275,97.227723,0.073245,93.069307,95.594059
9,MCK,276.38,7236,0.562599,94.851485,0.368841,95.049505,0.209333,95.841584,0.075955,93.267327,94.752475


## Format Excel Output

In [17]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

In [18]:
font_color = '#000000'
background_color = '#FFFFFF'

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format':'0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [19]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Number of Shares to Buy', integer_format],
                    'D': ['One-Year Price Return', percent_format],
                    'E': ['One-Year Return Percentile', percent_format],
                    'F': ['Six-Month Price Return', percent_format],
                    'G': ['Six-Month Return Percentile', percent_format],
                    'H': ['Three-Month Price Return', percent_format],
                    'I': ['Three-Month Return Percentile', percent_format],
                    'J': ['One-Month Price Return', percent_format],
                    'K': ['One-Month Return Percentile', percent_format],
                    'L': ['HQM Score', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_format)
    
writer.save()